In [104]:
# Load modules
import cyvcf2
import tqdm
import itertools
import pandas as pd

In [110]:
# Open text file with samples, populations, and habitats
txt_path = "./pop_map3_all_dist.txt"

# Initialize dictionaries to store samples in each habitat, and habitat of each population
pop_dict = {}
habitat_dict = {}

# Parse text file and add populations, sample, and habitats to dictionaries
with open(txt_path, "r") as fin:
    lines = fin.readlines()
    for l in lines:
        sl = l.split("\t")  # Splits each tab-delimited line
        sample = sl[0]  # Sample is first list element
        pop = sl[1]
        hab = sl[2].strip() # Strip removes newline character at end of line
        if pop in pop_dict.keys():
            pop_dict[pop].append(sample)
            habitat_dict[pop].append(hab)
        else:
            pop_dict[pop] = [sample]
            habitat_dict[pop] = [hab]


In [116]:
# Remove duplicate habitats for populations with more than one sample
for k, v in habitat_dict.items():
    habitat_dict[k] = list(set(v))

{'MW067': ['Urban'],
 'MW079': ['Urban'],
 'MW040': ['Urban'],
 'MW047': ['Rural'],
 'MW002': ['Urban'],
 'MW012': ['Urban'],
 'MW013': ['Urban'],
 'MW018': ['Rural'],
 'MW030': ['Urban'],
 'MW020': ['Rural'],
 'MW028': ['Urban'],
 'MW019': ['Rural'],
 'MW004': ['Urban'],
 'MW021': ['Rural'],
 'MW017': ['Rural'],
 'MW051': ['Urban'],
 'MW007': ['Rural'],
 'MW056': ['Urban'],
 'MW057': ['Urban'],
 'MW027': ['Urban'],
 'MW049': ['Rural'],
 'MW077': ['Urban'],
 'MW039': ['Urban'],
 'MW001': ['Urban'],
 'MW006': ['Rural'],
 'MW072': ['Urban'],
 'MW023': ['Rural'],
 'MW036': ['Rural'],
 'MW062': ['Urban'],
 'MW045': ['Rural'],
 'MW016': ['Urban'],
 'MW046': ['Rural'],
 'MW061': ['Urban'],
 'MW068': ['Urban'],
 'MW029': ['Urban'],
 'MW014': ['Urban'],
 'MW042': ['Urban'],
 'MW003': ['Urban'],
 'MW031': ['Urban'],
 'MW015': ['Urban'],
 'MW009': ['Rural'],
 'MW011': ['Urban'],
 'MW064': ['Urban'],
 'MW069': ['Urban'],
 'MW035': ['Rural'],
 'MW008': ['Rural'],
 'MW005': ['Rural'],
 'MW074': ['U

### Pooled Urban-rural Fst

In this section we estimated Fst pooling all urban and rural samples together

In [69]:
# Parse text files above to get all urban and rural samples
urban_samples = []
rural_samples = []
with open(txt_path, "r") as fin:
    lines = fin.readlines()
    for l in lines:
        sl = l.split("\t")
        sample = sl[0]
        pop = sl[1]
        hab = sl[2].strip()
        if hab == "Urban":
            urban_samples.append(sample)
        elif hab == "Rural":
            rural_samples.append(sample)
urban_samples 

['1.2_1A10_sorted',
 '1.2_1A11_sorted',
 '1.2_1A1_sorted',
 '1.2_1A3_sorted',
 '1.2_1A4_sorted',
 '1.2_1A5_sorted',
 '1.2_1A8_sorted',
 '1.2_1B10_sorted',
 '1.2_1B11_sorted',
 '1.2_1B2_sorted',
 '1.2_1B3_sorted',
 '1.2_1B6_sorted',
 '1.2_1B8_sorted',
 '1.2_1B9_sorted',
 '1.2_1C10_sorted',
 '1.2_1C1_sorted',
 '1.2_1C3_sorted',
 '1.2_1C5_sorted',
 '1.2_1C6_sorted',
 '1.2_1C8_sorted',
 '1.2_1D10_sorted',
 '1.2_1D11_sorted',
 '1.2_1D2_sorted',
 '1.2_1D3_sorted',
 '1.2_1D6_sorted',
 '1.2_1D7_sorted',
 '1.2_1D8_sorted',
 '1.2_1E2_sorted',
 '1.2_1E3_sorted',
 '1.2_1E6_sorted',
 '1.2_1E8_sorted',
 '1.2_1F10_sorted',
 '1.2_1F2_sorted',
 '1.2_1F4_sorted',
 '1.2_1F5_sorted',
 '1.2_1F7_sorted',
 '1.2_1F8_sorted',
 '1.2_1F9_sorted',
 '1.2_1G10_sorted',
 '1.2_1G2_sorted',
 '1.2_1G5_sorted',
 '1.2_1G6_sorted',
 '1.2_1G8_sorted',
 '1.2_1G9_sorted',
 '1.2_1H10_sorted',
 '1.2_1H1_sorted',
 '1.2_1H2_sorted',
 '1.2_1H3_sorted',
 '1.2_1H5_sorted',
 '1.2_1H7_sorted',
 '1.2_1H8_sorted',
 '1.2_1H9_sorted',
 '

In [79]:
# Load urban and rural VCFs
urban_vcf = cyvcf2.VCF("populations.snps.vcf", samples = urban_samples)
rural_vcf = cyvcf2.VCF("populations.snps.vcf", samples = rural_samples)

In [49]:
# Create dictionaries with SNP positions as keys and alternative allele frequencies as values
# Do this for both urban and rural populations
urban_af_dict = {f"{snp.CHROM}_{snp.POS}": [snp.aaf] for snp in urban_vcf}
rural_af_dict = {f"{snp.CHROM}_{snp.POS}": [snp.aaf] for snp in rural_vcf}

In [82]:
n_urban = len(urban_samples)
n_rural = len(rural_samples)
num = []
denom = []
for snp, af in urban_af_dict.items():
    u_af = af[0]  # Allele frequency in urban population
    r_af = rural_af_dict[snp][0]  # Allele frequency at same site in rural population

    # Hudson's Fst
    a = (u_af - r_af) ** 2
    b = (u_af * (1 - u_af)) / (n_urban - 1)
    c = (r_af * (1 - r_af)) / (n_rural - 1)
    d = (u_af * (1 - r_af))
    e = (r_af * (1 - u_af))

    numerator = a - b - c
    num.append(numerator)
    denominator = d + e
    denom.append(denominator)
fst = sum(num) / sum(denom)  # Ratio of averages approach from Bhatia 2013
print(fst)

### Fst for all pairwise combinations

Estimate Fst for all pariwise combinations of populations, provided both have more than 1 sample

In [120]:
# Create list with pairwise combinations of populations
pop_comb = list(itertools.combinations(pop_dict.keys(), 2))

# Create dictionary to store results
results = {"pop1": [], "pop2": [], "pop1_hab": [], "pop2_hab": [], "fst": []}

# Iterate over pairwise combinations
for comb in tqdm.tqdm(pop_comb):

    # Pull out population IDs and get sample lists
    pop1 = comb[0]
    pop2 = comb[1]

    pop1_samples = pop_dict[pop1]
    pop2_samples = pop_dict[pop2]
    n_pop1 = len(pop1_samples)
    n_pop2 = len(pop2_samples)

    if n_pop1 > 1 and n_pop2 > 2:
        
        results["pop1"].append(pop1)
        results["pop2"].append(pop2)
        pop1_hab = habitat_dict[pop1][0]
        pop2_hab = habitat_dict[pop2][0]
        results["pop1_hab"].append(pop1_hab)
        results["pop2_hab"].append(pop2_hab)

        # Load VCFs for pop1 and pop2 samples
        pop1_vcf = cyvcf2.VCF("populations.snps.vcf", samples = pop1_samples)
        pop2_vcf = cyvcf2.VCF("populations.snps.vcf", samples = pop2_samples)
    
        # Get allele frequencies for each population
        pop1_af_dict = {f"{snp.CHROM}_{snp.POS}": [snp.aaf] for snp in pop1_vcf}
        pop2_af_dict = {f"{snp.CHROM}_{snp.POS}": [snp.aaf] for snp in pop2_vcf}
    

        num = []
        denom = []
        
        for snp, af in pop1_af_dict.items():
            
            pop1_af = af[0]
            pop2_af = pop2_af_dict[snp][0]
            
            a = (pop1_af - pop2_af) ** 2
            b = (pop1_af * (1 - pop1_af)) / (n_pop1 - 1)
            c = (pop2_af * (1 - pop2_af)) / (n_pop2 - 1)
            d = (pop1_af * (1 - pop2_af))
            e = (pop2_af * (1 - pop1_af))
        
            numerator = a - b - c
            num.append(numerator)
            denominator = d + e
            denom.append(denominator)
    
        fst = sum(num) / sum(denom)
        results["fst"].append(fst)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7381/7381 [04:07<00:00, 29.79it/s]


In [121]:
fst_df = pd.DataFrame(results)
fst_df.to_csv("./fst.csv", index = False)

In [ ]:
df %>% mutate(fst = ifelse(fst < 0, 0, fst)) %>% group_by(pop1_habitat, pop2_habitat) %>% summarize(mean = mean(fst))